In [31]:
import seaborn as sns
from Bio.Phylo._utils import draw
import matplotlib.pyplot as plt
from src.distribution_analysis.process_tree import get_observed_nodes

import seaborn as sns
from src.distribution_analysis.process_tree import get_observed_nodes, get_clade_split_df
from src.datasets.load_trees import load_trees, TreeDataset
from src.utils.tree_utils import get_taxa_names
import pandas as pd
from tqdm import tqdm

from src.distribution_analysis.clade import ObservedCladeSplit

from Bio.Phylo.BaseTree import Tree, Clade

In [2]:
import warnings
warnings.filterwarnings('ignore')

sns.set_style("whitegrid")

In [3]:
DATA_SET = TreeDataset.YULE_10
DATA_SET_NAME = DATA_SET.value

In [4]:
trees = load_trees(DATA_SET, max_trees=50_000)

2it [00:10,  5.44s/it]


In [5]:
taxa_names = get_taxa_names(trees[0])

In [46]:
df_dict = {
    "min_branch_0": [],
    "min_branch_1": [],
}

def process_tree(clade: Clade):
    if clade.is_terminal():
        return
    
    process_tree(clade.clades[0])
    process_tree(clade.clades[1])

    min_clade_idx = 0 if clade.clades[0].branch_length < clade.clades[1].branch_length else 1
    
    if not clade.clades[min_clade_idx].is_terminal():
        min_min_clade_idx = 0 if clade.clades[min_clade_idx].clades[0].branch_length < clade.clades[min_clade_idx].clades[1].branch_length else 1
        df_dict["min_branch_0"].append(clade.clades[min_clade_idx].branch_length)
        df_dict["min_branch_1"].append(clade.clades[min_clade_idx].clades[min_min_clade_idx].branch_length)

for tree in trees:
    process_tree(tree.root)

In [47]:
from scipy.stats import pearsonr
pearsonr(df_dict["min_branch_0"], df_dict["min_branch_1"])

PearsonRResult(statistic=np.float64(0.44562859182455034), pvalue=np.float64(0.0))

In [54]:
df_dict = {
    "branch_l": [],
    "branch_ll": [],
    "branch_lr": [],
}

def process_tree(clade: Clade):
    if clade.is_terminal():
        return

    df_dict["branch_l"].append(clade.branch_length)
    df_dict["branch_ll"].append(clade.clades[0].branch_length)
    df_dict["branch_lr"].append(clade.clades[1].branch_length)

    process_tree(clade.clades[0])
    process_tree(clade.clades[1])

for tree in trees:
    process_tree(tree.root)

from scipy.stats import pearsonr
print(pearsonr(df_dict["branch_l"], df_dict["branch_ll"]))
print(pearsonr(df_dict["branch_l"], df_dict["branch_lr"]))

PearsonRResult(statistic=np.float64(-0.04615891266974981), pvalue=np.float64(9.789518001162844e-211))
PearsonRResult(statistic=np.float64(0.11669281039455803), pvalue=np.float64(0.0))
